# Swapping out `np.X` for `jnp.X`

XLA, their Accelerated Linear Algebra library is a compiler that takes generated computation graphs and compiles them into low-level machine instructions.

In [ ]:
import numpy as np
from typing import TypeAlias
import time

from hyperparameters import (
    _dt,
    _t_max,
    _tau_m,
    _V_reset,
    _V_thresh,
    _R,
    num_simulations
)
import jax.numpy as jnp
from tqdm.notebook import tqdm

with open('weights.npy', 'rb') as f:
    W = np.load(f)

# Initial conditions
n_neurons = len(W)# Number of neurons in the network
_V = jnp.ones(n_neurons) * _V_reset  # Initial potentials

# Type Definitions for Clarity

In [ ]:
Tensor3D: TypeAlias = jnp.ndarray
Mat: TypeAlias = jnp.ndarray
Vec: TypeAlias = jnp.ndarray 

# Run the Simulations

In [ ]:
def run_simulation(
    W: Mat,
    V: Vec,

    # Neuron Parameters
    tau_m: float,
    v_reset: float,
    v_thresh: float,
    membr_R: float,

    # How long do we run for? 
    t_max: float,
    dt: float, 

):
    # Simulation

    spike_train = []
    for i, t in enumerate(np.arange(0, t_max, dt)):
        if i == 0:
            continue

        fired = V >= v_thresh
        V = jnp.where(fired, v_reset, V)
        
        # Record spike times
        spike_train.append(fired)
    
        # Update voltages
        I_syn = W.dot(spike_train[-1])  # Synaptic current from spikes
        dV = (dt / tau_m) * (-V + v_reset + membr_R * I_syn)
        V = V + dV
    
        # No self-inputs; neurons cannot spike themselves in this timestep
        V = jnp.where(fired, v_reset, V)
    return spike_train

time_arr = []
for i in range(num_simulations):
    start = time.time()
    spike_train = run_simulation(
        W,
        _V,
        _tau_m, _V_reset, _V_thresh, _R,
        _t_max, _dt
    )
    end = time.time()
    print(f"Iteration {i} took: {end - start} seconds")
    time_arr.append(end - start)

print(f"Average Time: {np.mean(time_arr)}")
print(f"S.Dev Time: {np.std(time_arr)}")